In [35]:

import pandas as pd
import numpy as np

sad_main = pd.read_csv('myfolder/sad_main.csv', parse_dates=['date'])
sad_main['id'] = sad_main['id'].apply(lambda x: f"sad_{x}")
sad_emoji = pd.read_csv('myfolder/sad_emoji.csv')
sad_emoji['id'] = sad_emoji['id'].apply(lambda x: f"sad_{x}")
sad_emotion = pd.read_csv('StudentsAgainstDiscrimination_emotion.csv')
sad_emotion['id'] = sad_emotion['id'].apply(lambda x: f"sad_{x}")


aqd_main = pd.read_csv('myfolder/aqd_main.csv', parse_dates=['date'])
aqd_main['id'] = aqd_main['id'].apply(lambda x: f"aqd_{x}")
aqd_emoji = pd.read_csv('myfolder/aqd_emoji.csv')
aqd_emoji['id'] = aqd_emoji['id'].apply(lambda x: f"aqd_{x}")
aqd_emotion = pd.read_csv('antiquotadu_emotion.csv')
aqd_emotion['id'] = aqd_emotion['id'].apply(lambda x: f"aqd_{x}")


pusab_main = pd.read_csv('myfolder/pusab_main.csv', parse_dates=['date'])
pusab_main['id'] = pusab_main['id'].apply(lambda x: f"pusab_{x}")
pusab_emoji = pd.read_csv('myfolder/pusab_emoji.csv')
pusab_emoji['id'] = pusab_emoji['id'].apply(lambda x: f"pusab_{x}")
pusab_emotion = pd.read_csv('pusab_emotion.csv')
pusab_emotion['id'] = pusab_emotion['id'].apply(lambda x: f"pusab_{x}")


combined_main = pd.concat([sad_main, aqd_main, pusab_main], ignore_index=True)
combined_emoji = pd.concat([sad_emoji, aqd_emoji, pusab_emoji], ignore_index=True)
combined_emotion = pd.concat([sad_emotion, aqd_emotion, pusab_emotion], ignore_index=True)

In [60]:
emotion_list = ['Happiness', "Anger", "Sadness", "Fear", "Disgust", "Surprise"]


combined_emotion_analysis = combined_emotion.merge(combined_main[['id', 'date']], on='id', validate='one_to_one')

combined_emotion_analysis['date'] = combined_emotion_analysis['date'].dt.date
combined_emotion_analysis['dataset'] = combined_emotion_analysis['id'].str.split("_").apply(lambda x: x[0]).map({'aqd':"antiquotadu","sad":'StudentsAgainstDiscrimination', 'pusab':'pusab'})
combined_emotion_analysis['ekman_emotion'] = combined_emotion_analysis['ekman_emotion'].replace({'Joy':"Happiness", "anger":"Anger"}).apply(lambda x: x if x in emotion_list else np.nan)

combined_emotion_analysis = pd.concat([combined_emotion_analysis, combined_emotion_analysis.assign(dataset='Combined')], ignore_index=True)



# combined_emotion[combined_emotion['ekman_emotion'].isin(emotion_list)]['ekman_emotion'].value_counts()
emotion_show_df = combined_emotion_analysis.drop(columns=['id']).groupby(['dataset', 'date', 'ekman_emotion'], dropna=False)['ekman_emotion'].count().rename('counts').reset_index()


emotion_show_df['Percentage'] = emotion_show_df.groupby(['dataset', 'date'])['counts'].transform(lambda x: x/x.sum()*100)


In [25]:
# sad_main['date'].dtype
combined_emoji_analysis = combined_emoji.merge(combined_main[['id', 'date']], on='id', validate='one_to_one')

combined_emoji_analysis['date'] = combined_emoji_analysis['date'].dt.date
combined_emoji_analysis['dataset'] = combined_emoji_analysis['id'].str.split("_").apply(lambda x: x[0]).map({'aqd':"antiquotadu","sad":'StudentsAgainstDiscrimination', 'pusab':'pusab'})
combined_emoji_analysis = pd.concat([combined_emoji_analysis, combined_emoji_analysis.assign(dataset='Combined')], ignore_index=True)

my_set =set()
for i, j in combined_emoji_analysis.drop(columns='id').groupby(['dataset', 'date']):
    # print(i)
    # print(j.drop(columns=['date', 'dataset']).sum().sort_values(ascending=False)[:5])
    my_set.update(j.drop(columns=['date', 'dataset']).sum().sort_values(ascending=False)[:5].index)

emoji_show_df = combined_emoji_analysis.drop(columns='id').groupby(['dataset', 'date'])[list(my_set)].sum().fillna(0).astype(int).melt(value_name="count", var_name="emoji", ignore_index=False).reset_index()

def foo(x):
    temp = x.sum()
    return temp/temp.sum() * 100

emoji_perc = combined_emoji_analysis.drop(columns='id').groupby(['dataset', 'date']).apply(foo, include_groups=False)[list(my_set)].fillna(0).melt(value_name="Percentage", var_name="emoji", ignore_index=False).reset_index()

# print(pusab_emoji.shape, pusab_main.shape, pusab_emotion.shape)
# combined_emoji.dropna(how='all').shape
# sad_emoji
# sad_main
# pd.read_csv('myfolder/sad_main.csv', parse_dates=['date'])['id']
# my_set

In [14]:
import plotly.express as px

fig = px.line(emoji_show_df, x='date', y='count', color='emoji', facet_row='dataset', height=1200, title="Count of Emoji for each day in channels")

fig.update_yaxes(matches=None)
fig.show()
# sns.relplot(data=emoji_show_df, x='date', y='count', hue='emoji', row='dataset')

In [26]:

fig = px.line(emoji_perc, x='date', y='Percentage', color='emoji', facet_row='dataset', height=1200, title="Percentage of each Emoji for each day in channels", range_y=[0,100])

# fig.update_yaxes(matches=None)
# fig.update_yaxes()
fig.show()

In [62]:

fig = px.line(emotion_show_df.dropna(), x='date', y='counts', color='ekman_emotion', facet_row='dataset', height=1200, title="Count of Ekman emotion for each day in channels")

fig.update_yaxes(matches=None)
fig.show()

In [64]:

fig = px.line(emotion_show_df.dropna(), x='date', y='Percentage', color='ekman_emotion', facet_row='dataset', height=1200, title="Percentage of each emotion for each day in channels", range_y=[0,100])

# fig.update_yaxes(matches=None)
# fig.update_yaxes()
fig.show()

In [28]:
# pd.concat([sad_main, aqd_main, pusab_main], ignore_index=True)
# pd.concat([sad_emoji, aqd_emoji, pusab_emoji], ignore_index=True)
# pd.concat([sad_emotion, aqd_emotion, pusab_emotion], ignore_index=True)
# combined_emoji_analysis['date'].nunique()
# help(fig.update_yaxes)
combined_emotion['ekman_emotion'].value_counts()

ekman_emotion
Happiness                                                                                           1883
Anger                                                                                               1474
Sadness                                                                                              720
Fear                                                                                                 378
Disgust                                                                                              134
Surprise                                                                                              92
Neutral                                                                                                2
anticipation                                                                                           1
Joy                                                                                                    1
neutral                                  

In [111]:
print(combined_main.groupby(combined_main['date'].dt.date)['text_plain'].agg(lambda x: "\n\n\n".join(x)).iloc[2])

media_image



media_image



media_video
আজ চট্টগ্রামে কোটা সংস্কারের দাবিতে আন্দোলনরত সাধারণ শিক্ষার্থীদের উপর এভাবে প্রকাশ্যে অস্ত্রধারী সন্ত্রাসীরা হামলা চালালেও প্রশাসন ছিল নির্বিকার।


media_video
ঢাবির রোকেয়া হলে ছাত্রলীগ নেত্রীকে আপ্যায়নের জন্য প্রস্তুত সাধারণ শিক্ষার্থীরা


media_video
এভাবেই ঢাবির মুহসিন হলে বীরের বেশে প্রবেশ করে সাধারণ শিক্ষার্থীরা।


media_video
ঢাবির রোকেয়া হলে বাঘিনীদের গর্জন। ছাত্রলীগ নেত্রীদের হলছাড়া করা হয়েছে। হলগুলো এখন শিক্ষার্থীদের নিয়ন্ত্রণে।


media_video



media_video



media_image
নতুন কর্মসূচি ঘোষণা


media_image
রাজশাহী বিশ্ববিদ্যালয় সাধারণ ছাত্র-ছাত্রীদের দখলে যাওয়ার পর থেকেই সোশ্যাল মিডিয়ায় নানানভাবে আতংক সৃষ্টির চেষ্টায় রয়েছে ছাত্রলীগ।


media_video
রাজধানীর শেরে বাংলা কৃষি বিশ্ববিদ্যালয় (শেকৃবি) ক্যাম্পাসের ভিডিও। সাধারণ শিক্ষার্থীদের "ভুয়া-ভুয়া" শ্লোগান শুনে বন্ধ হয়ে গেল ছাত্রলীগের শ্লোগান!


ঘোষণা 🚨📢📣

আজকে পুলিশ ও ছাত্রলীগের যৌথ হামলা ও গুলিবর্ষণে শহীদ ভাইদের জন্য আগামীকাল ১৭ জুলাই দুপুর ২:০০ টায় রাজু ভাষ্কর্যের পাদদেশে "গায়েবানা জানা

In [69]:

from openai import OpenAI
from dotenv import load_dotenv

In [70]:

client = OpenAI()

In [72]:

a = client.batches.retrieve('batch_68690477ef788190a1452cbe5de8956d')

In [93]:
# str(a)
a.output_file_id

'file-V4eCq4y5LLe9nXsyd3wEaU'

In [78]:
from openai import OpenAI
from dotenv import load_dotenv
import json
import io
from PIL import Image, ImageOps
import base64

load_dotenv()

def count_openai_embedding_tokens(text: str) -> int:
    """Counts the number of tokens in a text string for OpenAI embedding models."""
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(text))
    return num_tokens


system_prompt = """You are an expert emotion analysis AI. Your task is to analyze the provided Telegram post and identify the **primary Ekman emotion** expressed.

The Ekman emotions you must choose from are:
- **Anger**: Feeling of antagonism provoked by ill-treatment or opposition.
- **Disgust**: Feeling of revulsion or strong disapproval aroused by something unpleasant or offensive.
- **Fear**: Unpleasant emotion caused by the belief that someone or something is dangerous, likely to cause pain, or a threat.
- **Happiness**: State of well-being and contentment; joy.
- **Sadness**: Emotional pain associated with, or characterized by, feelings of disadvantage, loss, despair, grief, helplessness, disappointment, and sorrow.
- **Surprise**: Feeling caused by something unexpected or unusual.

Please analyze the following telegram post and provide the primary Ekman emtion.

**If the primary emotion doesn't match any of the Ekman emotion then please provide the closest emotion outside the Ekman classification.

If no emotion could be found at all, then please return 'NA'."""

# system_prompt = """You are an expert emotion analysis AI. Your task is to analyze the provided Telegram post and identify the **primary Ekman emotion** expressed.

# The Ekman emotions you must choose from are:
# - **Anger**: Feeling of antagonism provoked by ill-treatment or opposition.
# - **Disgust**: Feeling of revulsion or strong disapproval aroused by something unpleasant or offensive.
# - **Fear**: Unpleasant emotion caused by the belief that someone or something is dangerous, likely to cause pain, or a threat.
# - **Happiness**: State of well-being and contentment; joy.
# - **Sadness**: Emotional pain associated with, or characterized by, feelings of disadvantage, loss, despair, grief, helplessness, disappointment, and sorrow.
# - **Surprise**: Feeling caused by something unexpected or unusual.

# Please analyze the following telegram post and return the dominant Ekman emotion in structured JSON format. If no emotion could be found, then please return 'N/A'. In addition to the emotion label, provide a short explanation for why this emotion was identified.

# Format your output as a valid JSON object with the following fields:
# {
#   "explanation": "<brief explanation (1-2 sentences) of which Ekman emotion is most dominant in the post>"
#   "emotion": "<one of: Anger, Disgust, Fear, Happiness, Sadness, Surprise, N/A>",
# }
# Ensure the output is strictly valid JSON."""


# response_format = {
#     "type": "json_schema",
#     "name": "ekman_emotion",
#     "schema": {
#         "type": "object",
#         "properties": {
#             "emotion": {
#                 "type": "string",
#                 "description": "The primary Ekman emotion expressed in the text.",
#                 "enum": [
#                     "Anger",
#                     "Disgust",
#                     "Fear",
#                     "Happiness",
#                     "Sadness",
#                     "Surprise",
#                     "NA",
#                 ],
#             }
#         },
#         "required": ["emotion"],
#         "additionalProperties": False,
#     },
#     "strict": True,
# }

tool = [
      {
        "type": "function",
        "function": {
          "name": "classify_ekman_emotion",
          "description": "Classify the dominant Ekman emotion from a Telegram post",
          "parameters": {
            "type": "object",
            "properties": {
              "explanation": {
                "type": "string",
                "description": "A brief explanation for which Ekman emotion is dominant"
              },
              "emotion": {
                "type": "string",
                # "enum": [
                #     "Anger",
                #     "Disgust",
                #     "Fear",
                #     "Happiness",
                #     "Sadness",
                #     "Surprise",
                #     "NA",
                # ],
                "description": "The dominant Ekman emotion"
              },
            },
            "required": ["explanation", "emotion"]
          }
        }
      }
    ]

my_texts = ["some text", "Doesn't look fun at all", "some other text", "This is so fun", "That is repulsive"]

# with open("my_test.jsonl", "w") as f:
#     for i, x in enumerate(my_texts):
#         json.dump(
#             {
#                 "custom_id": f"random_id{i}",
#                 "method": "POST",
#                 "url": "/v1/chat/completions",
#                 "body": {
#                     "model": "gpt-4.1-nano",
#                     "messages": [
#                         {"role": "system", "content": system_prompt},
#                         {"role": "user", "content": x},
#                     ],
#                 "tools":tool,
#                 "tool_choice":'required'
                
#                 },
#             },
#             f,
#         )
#         f.write("\n")

for i in range(0, len(my_texts), 2):
  
  with open(f"test_jsonl/antiquotadu_discuss_{i}_{i+2-1}.jsonl", "w") as f:
      for j, row in enumerate(my_texts[i:i+2]):
      # for i, x in enumerate(my_texts):
        #   if pd.isnull(row['image_address']):
        json.dump(
            {
                "custom_id": f"antiquotadu_discuss_{j}",
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model": "gpt-4.1-nano",
                    "messages": [
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": row},
                    ],
                "tools":tool,
                "tool_choice":'required'
                
                },
            },
            f,
        )
        #   else:
        #     img = Image.open(row['image_address'])
            
        #     min_size = min(img.size)
        #     buffer = io.BytesIO()
        #     if min_size > 128:
        #       img_small = ImageOps.scale(img, 128/min_size)
        #       img_small.save(buffer, format="jpeg")
        #     else:
        #       img.save(buffer, format='jpeg')



            # encoded_img = base64.b64encode(buffer.getvalue()).decode("utf-8")
            
            # json.dump(
            #     {
            #         "custom_id": f"antiquotadu_discuss_{row['id']}",
            #         "method": "POST",
            #         "url": "/v1/chat/completions",
            #         "body": {
            #             "model": "gpt-4.1-nano",
            #             "messages": [
            #                 {"role": "system", "content": system_prompt},
            #                 {"role": "user", "content": [{"type":"text", "text":row['Text_Emotion']},
            #                                             {"type":"image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded_img}"}}]},
            #             ],
            #         "tools":tool,
            #         "tool_choice":'required'
                    
            #         },
            #     },
            #     f,
            # )
        f.write("\n")

# client = OpenAI()


In [106]:
from glob import glob

filenames = []

for f in glob('discuss_aqd/*.jsonl'):
    filenames.append(f)

filenames

['discuss_aqd/antiquotadu_discuss_8000_8999.jsonl',
 'discuss_aqd/antiquotadu_discuss_30000_30999.jsonl',
 'discuss_aqd/antiquotadu_discuss_3000_3999.jsonl',
 'discuss_aqd/antiquotadu_discuss_14000_14999.jsonl',
 'discuss_aqd/antiquotadu_discuss_17000_17999.jsonl',
 'discuss_aqd/antiquotadu_discuss_1000_1999.jsonl',
 'discuss_aqd/antiquotadu_discuss_26000_26999.jsonl',
 'discuss_aqd/antiquotadu_discuss_19000_19999.jsonl',
 'discuss_aqd/antiquotadu_discuss_23000_23999.jsonl',
 'discuss_aqd/antiquotadu_discuss_12000_12999.jsonl',
 'discuss_aqd/antiquotadu_discuss_31000_31999.jsonl',
 'discuss_aqd/antiquotadu_discuss_16000_16999.jsonl',
 'discuss_aqd/antiquotadu_discuss_27000_27999.jsonl',
 'discuss_aqd/antiquotadu_discuss_22000_22999.jsonl',
 'discuss_aqd/antiquotadu_discuss_34000_34999.jsonl',
 'discuss_aqd/antiquotadu_discuss_0_999.jsonl',
 'discuss_aqd/antiquotadu_discuss_13000_13999.jsonl',
 'discuss_aqd/antiquotadu_discuss_6000_6999.jsonl',
 'discuss_aqd/antiquotadu_discuss_20000_20

In [107]:
import pandas as pd
import numpy as np

check_df = pd.DataFrame({'filenames':filenames, 'input_file_id':np.nan, 'batch_id':np.nan, 'output_file_id':np.nan}, dtype=str)
check_df.to_csv("check_df.csv", index=False)

In [101]:
import time

check_df = pd.read_csv('check_df.csv', dtype=str)
filenames = check_df['filenames'].tolist()
# client = OpenAI_stuff()


for file in filenames:
    row = check_df[check_df['filenames']==file].iloc[0]
    # print(row)
    # print(check_df)
# for _, row in check_df.iterrows():
    if pd.isnull(row['input_file_id']):
        # batch_input_file = client.files.create(
        #     file=open(row["filenames"], "rb"), purpose="batch"
        # )
        # batch_input_file_id = batch_input_file.id
        batch_input_file_id = file + "_input"
        check_df.loc[check_df['filenames']==file, 'input_file_id'] = batch_input_file_id
        check_df.to_csv('check_df.csv', index=False)
        row = check_df[check_df['filenames']==file].iloc[0]
        print(file + 'in')
    if pd.isnull(row['batch_id']):
        # batch_obj = client.batches.create(
        #     input_file_id=row['input_file_id'],
        #     endpoint="/v1/chat/completions",
        #     completion_window="24h",
        #     metadata={"description": f"Antiquotadu discuss emotion analysis_{file}"},
        # )
        # batch_id = batch_obj.id
        batch_id = file + "_batch"
        check_df.loc[check_df['filenames']==file, 'batch_id'] = batch_id
        check_df.to_csv('check_df.csv', index=False)
        row = check_df[check_df['filenames']==file].iloc[0]
        print(file + "batch")
    if pd.isnull(row['output_file_id']):
        # while True:
        #     batch_obj = client.batches.retrieve(row['batch_id'])
        #     if batch_obj.status == 'completed':
        #         break
        #     if batch_obj.status in ['failed', 'expired', 'cancelled']:
        #         raise Exception(str(batch_obj))
        #     time.sleep(3600)
        check_df.loc[check_df['filenames']==file, 'output_file_id'] = file + "output" # batch_obj.output_file_id
        check_df.to_csv('check_df.csv', index=False)
        print(file + "out")
        
        

        
    #     pass
    # elif pd.isnull()

In [99]:
check_df

,filenames,input_file_id,batch_id,output_file_id
0,test_jsonl/antiquotadu_discuss_2_3.jsonl,test_jsonl/antiquotadu_discuss_2_3.jsonl_input,test_jsonl/antiquotadu_discuss_2_3.jsonl_batch,test_jsonl/antiquotadu_discuss_2_3.jsonloutput
1,test_jsonl/antiquotadu_discuss_0_1.jsonl,test_jsonl/antiquotadu_discuss_0_1.jsonl_input,test_jsonl/antiquotadu_discuss_0_1.jsonl_batch,test_jsonl/antiquotadu_discuss_0_1.jsonloutput
2,test_jsonl/antiquotadu_discuss_4_5.jsonl,test_jsonl/antiquotadu_discuss_4_5.jsonl_input,test_jsonl/antiquotadu_discuss_4_5.jsonl_batch,test_jsonl/antiquotadu_discuss_4_5.jsonloutput
